# 단어 표현법

## 단어의 표현 방법
단어의 표현 방법은 크게 국소 표현(Local Representation)방법과 분산 표현(Distributed Representation)방법으로 나뉨

국소 표현(=이산 표현): 해당 단어 그 자체만보고, 특정 값을 맵핑하여 단어를 표현하는 방법
EX) '강아지', '귀여운', '사랑스런' 단어가 들어오면 각각 1, 2, 3 같이 숫자로 맵핑

분산 표현(=연속 표현): 그 단어를 표현하고자 주변을 참고하여 단어를 표현하는 방법
EX) '강아지'라는 단어 주변에는 주로 '귀여운', '사랑스런'이라는 단어가 자주 등장하므로, '강아지'라는 단어는 '귀여운', '사랑스런' 느낌이다로 단어를 정의

![nn](img/word_representation01.png)

## Bag of Words(BoW)

직역하면 단어들의 가방이라는 의미
단어들의 순서는 전혀 고려하지 않고, 단어들의 출현 빈도(frequency)에만 집중하는 텍스트 데이터 수치화 표현 방법

문서를 자동으로 분류하기 위한 방법중 하나로서, 글에 포함된 단어들의 분포를 보고 이 문서가 어떤 종류의 문서인지를 판단함
컴퓨터 비전에도 사용되었음

### 한계
**1.문맥 의미 반영 부족**
>순서를 고려하지 않기 때문에 문맥적인 의미가 무시됨

**2.희소 행렬 문제**
>문서는 대부분 서로 다른 단어로 구성되며 이에따라 문서마다 단어가 나타나지 않는 경우가 훨씬 많음
>따라서 대부분 행렬이 0으로 채워기게됨

## 문서 단어 행렬(Document-Term Matrix, DTM)

서로 다른 문서들의 BoW들을 결합한 표현 방법
행과 열을 반대로 선택하면 TDM이라고 부르기도 함
서로 다른 문서 비교 가능


### 표기법
문서 단어 행렬이란 다수의 문서에서 등장하는 각 단어들의 빈도를 행렬로 표현한 것
쉽게 생각하면 각 문서에 대한 BoW를 하나의 행렬로 만든 것으로 생각할 수 있으며, BoW와 다른 표현 방법이 아니라 BoW 표현을 다수의 문서에 대해서 행렬로 표현하고 부르는 용어

EX)
문서1: 먹고 싶은 사과
문서2: 먹고 싶은 바나나
문서3: 길고 노란 바나나 바나나
문서4: 저는 과일이 좋아요

![nn](img/DTM_01.png)

문서 단어 행렬은 문서들을 서로 비교할 수 있도록 수치화할 수 있음
불용어를 제거한다면 더 정제된 DTM을 만들 수 있음

### 한계
**1.희소 표현**
>대부분 값이 0으로 채워질 수 있음

**2.단순 빈도 수 기반 접근**
>영어를 예시로 불용어인 'the'가 어떤 문서이든 자주 등장할 수 밖에 없음 -> 'the'의 빈도가 높다해서 문서1, 2, 3을 유사한 문서로 판단해서는 안됨

>중요한 단어에는 가중치 부여 필요

## TF-IDF(Term Frequency-Inverse Document Frequency)
모든 문서에서 자주 쓰일 수 밖에 없는 단어들(불용어)이 중요하다 인식될 수 있음
개별 문서에서 자주 등장하는 단어에 높은 가중치를 부여하되, 모든 문서에서 자주 등장하는 단어에 대해서는 패널티를 부과하는 방법



tf(d, t): 특정 문서 d에서의 특정 단어 t의 등장 횟수

df(t): 특정 단어 t가 등장한 문서의 수

특정 단어가 각 문서, 또는 문서들에서 몇 번 등장했는지는  관심가지지 않으며 오직 특정 단어 t가 등장한 문서의 수에만 관심을 가짐
문서2, 문서3 에서 바나나가 각 100번 등장하더라도 df는 2임

idf(d, t): df(t)에 반비례하는 수

![nn](img/idf_01.png)

idf는 df의 역수이지만 단순히 역수로 나타내는 경우 총 문서의 수 n이 커질수록, idf값은 기하급수적으로 증가하게 됨

그렇기 때문에 log를 사용

분모의 1을 더해주어 단어가 등장하지 않아도 분모가 0이 되는것을 방지

**log 사용시**　　　　　　　　　　　　　　　　**log 미사용시**

<img src="img/idf_02.png" width="30%" height="30%"/>
<img src='img/idf_03.png' width="31%" height="31%"/>


## 실습

### Bag of Words

In [1]:
from konlpy.tag import Okt

okt = Okt()

def build_bag_of_words(document):
    # 온점 제거 및 형태소 분석
    document = document.replace('.', '')
    tokenized_document = okt.morphs(document)

    word_to_index = {}
    bow = []

    for word in tokenized_document:
        if word not in word_to_index.keys():
            word_to_index[word] = len(word_to_index)
            # bow에 전부 기본값 1을 넣음
            bow.insert(len(word_to_index) - 1, 1)
        else:
            # 재등장하는 단어의 인덱스
            index = word_to_index.get(word)
            # 재등장한 단어는 해당하는 인덱스의 위치에 1을 더함
            bow[index] = bow[index] + 1
    
    return word_to_index, bow

In [2]:
doc1 = "정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다."
vocab, bow = build_bag_of_words(doc1)
print('vocabulary :', vocab)
print('bag of words vector :', bow)

vocabulary : {'정부': 0, '가': 1, '발표': 2, '하는': 3, '물가상승률': 4, '과': 5, '소비자': 6, '느끼는': 7, '은': 8, '다르다': 9}
bag of words vector : [1, 2, 1, 1, 2, 1, 1, 1, 1, 1]


### TF-IDF

In [1]:
import pandas as pd
from math import log

docs = [
  '먹고 싶은 사과',
  '먹고 싶은 바나나',
  '길고 노란 바나나 바나나',
  '저는 과일이 좋아요'
]

vocab = list(set(w for doc in docs for w in doc.split()))
vocab.sort()

In [2]:
N = len(docs)

def tf(t, d):
    return d.count(t)

def idf(t):
    df = 0
    for doc in docs:
        df += t in doc
    return log(N/(df+1))

def tfidf(t,d):
    return tf(t, d) * idf(t)

In [3]:
result = []

for i in range(N):
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]
        result[-1].append(tf(t, d))

tf_ = pd.DataFrame(result, columns=vocab)

In [4]:
tf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0,0,0,1,0,1,1,0,0
1,0,0,0,1,1,0,1,0,0
2,0,1,1,0,2,0,0,0,0
3,1,0,0,0,0,0,0,1,1
